<a href="https://colab.research.google.com/github/spynob/MAIS_Assignments/blob/main/Assignment2_SPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 ― Classification

## 0. Introduction

In this second assigment, we will explore another cornerstone of machine learning: supervised classification. We will be specifically classifying IMDB movie reviews by their positive (1) or negative (-1) score. To do this, we will first pre-process the raw data by cleaning and turning each review into a vector. Then, we will explore and fine-tune our use of the following learning algorithms for classification: naive Bayes classifiers, support vector machines, and random forests. 

* [Question 1.1](#scrollTo=m0QxxH3KngAg)
* [Question 2.1](#scrollTo=6VWSBN37uXod)
* [Question 2.2](#scrollTo=CuW0ahvaJtOt)
* [Question 2.3](#scrollTo=co54Ubd5QJDN)
* [Question 3.1](#scrollTo=fN-dse1NBQnk)
* [Question 3.2](#scrollTo=wfCjr-JrEJya)
* [Question 3.3](#scrollTo=l1iGVZtkE5fF)
* [Question 4.1](#scrollTo=YKLBuWjmAKoJ)
* [Question 5.1](#scrollTo=Myn-42J9ACsH)
* [Question 5.2](#scrollTo=1LAqa9be_3vR)
* [Question 6.1](#scrollTo=C144CYOeYPca) [Optional] 
* [Question 6.2](#scrollTo=rnIbpGe-Z52z) [Optional] 
* [Question 6.3](#scrollTo=W6CdJGpjcK2r) [Optional] 



$% latex commands for later use$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\B}{\mathbb{B}}$
$\newcommand{\argmax}{\operatorname*{arg\ max}}$
$\newcommand{\given}{\; \vert \;}$



## 1. Importing Libraries and Data

For this assignment, we will be using a dataset of IMDB reviews. The data consists of a csv file where the first column is a string containing a user review and the second column specifies whether the review was positve (1) o negative (-1). First, we will import any libraries that we might use.

**Note:** You may use any library you would like unless specified otherwise. 

### Question 1.1 Importing Libraries

Keep on adding in the section below any modules you use as you are completing the assignment.

In [1]:
import csv
import random
### Answer starts here ###
from bs4 import BeautifulSoup
import collections
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
### Answer ends here ###font

Let's download the dataset:

In [2]:
!wget https://raw.githubusercontent.com/McGillAISociety/BootcampAssignmentDatasets/master/data/assignment2/train_reviews.csv

--2023-02-21 01:08:06--  https://raw.githubusercontent.com/McGillAISociety/BootcampAssignmentDatasets/master/data/assignment2/train_reviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33333854 (32M) [text/plain]
Saving to: ‘train_reviews.csv’

train_reviews.csv   100%[===================>]  31.79M   145MB/s    in 0.2s    

2023-02-21 01:08:07 (145 MB/s) - ‘train_reviews.csv’ saved [33333854/33333854]



And create a function to print a review:

In [3]:
def print_review(review, score):
  print('--------------- Review with score of {} ---------------'.format(score))
  print(review)
  print('------------------------------------------------------')
  print()

Let's load the data and see what the first 10 reviews look like:


In [4]:
with open('train_reviews.csv') as csv_file:
  csv_reader = csv.reader(csv_file)
  colnames = next(csv_reader)  # skip column names
  data = list(csv_reader)

for review, score in random.sample(data, 10):
  print_review(review, score)

--------------- Review with score of 1 ---------------
I saw this movie with my dad. I must have been pretty young, around 15. It was on Star Movies one afternoon.The movie started a bit vaguely, but you could tell those robbers were gathering up for a score. It really caught pace after the first half hour.<br /><br />All the actors are great, especially Blades and Lou Diamond. I Guess it's the ensemble, they just play so well together. I can watch this film anytime.I think it is the relative stupidity of the plot and the characters trying to deal with a very weird score. The jokes are not corny but they are subtle and extreme at the same time that make them so hilarious.<br /><br />A perfect comedy for a lazy afternoon.
------------------------------------------------------

--------------- Review with score of -1 ---------------
Whoever made this nonsense completely missed the point. Jane is a silly comic strip to titillate without being sleazy.<br /><br />This giant mess tries to be

## 2. Preprocessing
 We will be converting our data into a binary bag-of-words representation (Google "binary bag-of-words"). To do this, we will perform two steps beforehand.

### Question 2.1 Cleaning the train data
Create a function called `clean`, which takes a string and then:

 1. lower-case all words 
 2. only keeps letters and spaces
 

 We also need to get rid of [HTML tags](https://www.javatpoint.com/html-tags) as they do not hold valuable information for classifying the review. A quick Google search on removing HTML tags with `regular expressions` will show you how to do this! 
  
  For example, the following review...
  
  >`This was the WORST movie I have EVER SEEN!! <br/>`
  
  ...will be cleaned to become:
  
  >`this was the worst movie i have ever seen`
  
   Of course, you could do more pre-processing steps if you would like, such as lemmatization, stemming, etc... but TOTALLY OPTIONAL! 

In [25]:
def clean(review):
  ### Start of Answer ###
  review = review.lower()
  return BeautifulSoup(review, "lxml").text
  ### End of Answer ###


Test your function with this example string

In [29]:
print(clean("This was the WORST movie I have EVER SEEN!! <br/> "))

this was the worst movie i have ever seen!!  


Now, we'll use the function to clean the whole dataset. We'll also turn the scores from strings to integers while we're at it.

In [30]:
X_train = []
y_train = []
for review, score in data:
  X_train.append(clean(review))
  y_train.append(int(score))

### Question 2.2 Picking features

We now need to turn each review into vectors. We will pick the 10,000 most recurring words in the train set as features

Using those 10,000 features, create a function called `vectorize` which will take a string as an input, and convert it to a vector using the binary bag of words representation.

For example, the string `"This movie made me cry"` will become a vector of size 10,000 with 5 elements being 1 (assuming each word is part of the 10,000 most common) and 9995 being 0, that it, is i will look something like

 > `[0, 0, ..., 0, 1, 0, ..., 0, 1, 0..., 0, 1, 0, ..., 0, 1, 0 ..., 0, 1, 0, ..., 0, 0]`
 
 In order to accomplish this task, you will
 
 1. write a `get_vocab` function which takes as an argument a list of (cleaned) reviews and the vocabulary size and outputs the a list of size `vocab_size` containing the most common words.
 2. write a `vectorize` function which takes as an argument a review and the vocabulary and turns the review into its binary bag of words representation.
 3. use the `vectorize` function to create a new variable called `X_train_vect` which will contain the bag-of-words representation of each data point contained in the `X_train` variable rather than its string representation.

**Note**: each step should take no longer than 30 seconds to run! Make sure you implement the functions efficiently, taking the time complexity of your solution into account. `collections.Counter` may be of use to you (see [here](https://realpython.com/python-dicts/) for an explanation of python dictionaries and [here](https://docs.python.org/3/library/collections.html#collections.Counter) for the documentation for `Counter`).

In [31]:
from typing import Collection
def get_vocab(reviews, vocab_size):
  ### Answer starts here ###
  c = collections.Counter()
  for review in reviews:
    #review = [j for i in review for j in i.split]
    review = review.split()
    c.update(review)
  vocabTup = c.most_common(vocab_size)
  vocabList = []
  for tup in vocabTup:
    vocabList.append(tup[0])
  return vocabList
  ### Answer ends here ###

In [32]:
import math
def power(x,n):
  return math.pow(x,n)
  

Test your function with the following code. The `vocabulary` variable should have a length of 10,000 and the most common words should be "the", "and", "a", etc.

In [33]:
num_features = 10000
vocabulary = get_vocab(X_train, num_features)
print(len(vocabulary))
print(vocabulary)

10000
['the', 'a', 'and', 'of', 'to', 'is', 'in', 'i', 'this', 'that', 'it', 'was', 'as', 'for', 'with', 'but', 'on', 'movie', 'his', 'are', 'not', 'film', 'you', 'have', 'he', 'be', 'at', 'one', 'by', 'an', 'they', 'from', 'all', 'who', 'like', 'so', 'just', 'or', 'has', 'her', 'about', "it's", 'some', 'if', 'out', 'what', 'very', 'when', 'there', 'more', 'she', 'would', 'even', 'good', 'my', 'only', 'their', 'no', 'really', 'had', 'which', 'can', 'up', 'were', 'see', 'than', 'we', '-', 'been', 'into', 'get', 'will', 'much', 'story', 'because', 'most', 'how', 'other', 'also', 'first', 'its', 'time', 'do', "don't", 'me', 'great', 'people', 'could', 'make', 'any', 'after', 'made', 'then', 'bad', 'think', 'being', 'many', 'him', 'never', 'two', 'too', 'little', 'where', 'well', 'way', 'watch', 'your', 'it.', 'did', 'does', 'them', 'best', 'movie.', 'know', 'seen', 'love', 'characters', 'character', 'movies', 'these', 'ever', 'still', 'over', 'should', 'films', 'such', 'plot', 'acting', '

In [34]:
def vectorize(review_string, vocab):
  ### Answer starts here ###
  vector = [0 for i in vocab]
  words = collections.Counter(review_string.split())
  for i in range(len(vocab)):
    if words[vocab[i]] != 0:
      vector[i] = 1
  return vector
  ### Answer ends here ###

Test your function with the following input. The vector should have four "1"s.

In [35]:
vector = vectorize("the and a of zyxw", vocabulary)
print(vector)
print(sum(vector))

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Now, vectorize the whole dataset. 

In [36]:
### Answer starts here ###
X_train_vect = []
for review in X_train:
  X_train_vect.append(vectorize(review, vocabulary))

### Answer ends here ###

In [37]:
for i in range(5):
  print_review(X_train_vect[i], y_train[i])

--------------- Review with score of -1 ---------------
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

For convenience, we will write a function called `preprocess_sample_point` which takes as input a single raw review and ouputs its binary bag-of-words representation.

In [38]:
def preprocess_sample_point(review, vocab):
  return vectorize(clean(review), vocab)

In [39]:
vectorized_review = preprocess_sample_point(
    'The movie was not bad, it was really good!', vocabulary)
print(sum(vectorized_review))
print(vectorized_review)

8
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### Question 2.3 Preparing the test set

Now that we have defined a cleaning function and extracted the features from the train set, we are ready to preprocess the test set. Implement the `preprocess` function below such that it:

1. Loads the raw data from a csv file 
2. Cleans and vectorizes the reviews
3. Converts the scores to `int`
4. Returns the data into a  `(X_test, y_test)` tuple

In [40]:
def preprocess(csv_filename, vocab):
  ### Answer starts here ###
  with open(csv_filename) as csv_file:
    csv_reader = csv.reader(csv_file)
    colnames = next(csv_reader)  # skip column names
    data = list(csv_reader)
  
  X = []
  y = []
  for review, score in data:
    X.append(vectorize(clean(review), vocab))
    y.append(int(score))

  return (X, y)
  ### Answer ends here ###

In [41]:
!wget https://raw.githubusercontent.com/McGillAISociety/BootcampAssignmentDatasets/master/data/assignment2/test_reviews.csv

--2023-02-21 01:20:31--  https://raw.githubusercontent.com/McGillAISociety/BootcampAssignmentDatasets/master/data/assignment2/test_reviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32551850 (31M) [text/plain]
Saving to: ‘test_reviews.csv’

test_reviews.csv    100%[===================>]  31.04M  --.-KB/s    in 0.1s    

2023-02-21 01:20:32 (237 MB/s) - ‘test_reviews.csv’ saved [32551850/32551850]



In [42]:
X_test, y_test = preprocess('test_reviews.csv', vocabulary)

In [43]:
for i in range(5):
  print_review(X_test[i], y_test[i])

--------------- Review with score of -1 ---------------
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


##3. Naive Bayes
Later in this assignment, we will use pre-existing implementations of two types of models: random forests and support vector machines. However, we will start first by implementing from sratch a third type of classifier: naive Bayes classifers. 

Naive Bayes classifiers are part of a larger family of classifiers which are called 'probabilistic classifiers': not only do they try to predict classes given features, but they also estimate probability distributions over a set of classes.

First, let's will go over some definitions:

**Definition:** A *prior probability* is the likelihood of an event given no further assumptions. For instance, the probability that it's raining is relatively low.

**Definiton:** A *posterior probability* or *conditional probability* is the likelihood of an event given that some other event has occurred. For instance, the probability that it's raining given that there are clouds is higher than if we don't make that assumption.

Now we will go over some motivation:

For the purpose of argument, imagine we had access to the probability distribution $\Pr$. That is, we know how likely features and classes are. For example, $\Pr(x_1 = 1)$ is the probability that the most common word, i.e. "the", is in a random movie review. Presumably, this probability is relatively high. As a second example, $\Pr(y = 1)$ is the probability that a random movie review is 'good'. 

Since we hypothetically have access to the whole probability distribution, we also know conditional probabilities. For instance, we would know $\Pr(y = -1 \; \vert \; x_1 = 0)$, which is the probability that a random review is 'bad', given that it does not contain the word "the".

Given a probability distribution, we can find an optimal classifier which simply picks the class which maximizes the probability that we will see that class given the observed features, in other words our classifier $f: \B^n \to \B$ is given by:

$$ f(x_1, \ldots, x_n) = \argmax_{c \in \B} \Pr(y = c \given x_1, \ldots, x_n ).$$

Where $\argmax$ returns the element in $\B$ which maximizes the expression to its right, and $\B$ is the set with two elements, $\{-1, 1\}$. For example, we have
$$ \argmax_{x \in \R} (x - x^2) = \frac 1 2, $$
since $\frac 1 2$ maximizes the expression $x - x^2$.

It would be great if we had access to the probability distribution $\Pr$, but unfortunately we don't in almost every case. This means we wish to try to estimate it given some samples, i.e. the training data.

However, we run into another issue: estimating the probability distribution is computionally expensive. Therefore, we assume that the different features are independent from one another. This is called the *naive conditional independence assumption*. In other words, we assume that

$$ \forall i \in \{1, \ldots, n\} : \Pr (x_i \given y, x_1, \ldots, x_{i-1}, x_{i+1}, \ldots, x_n) = \Pr(x_i \given y).$$

Using Bayes' Theorem, we can simplify the conditional independence assumption to:

$$\Pr(y \given x_1, \ldots, x_n) = \frac{\Pr(y) \prod_{i=1}^n \Pr(x_i \given y)}{\Pr(x_1, \ldots, x_n)}.$$

However, we can observe that the denominator is constant for a given input, so it's not actually necesarry to estimate it if all we want is to find the class with the maximum posterior probability. In other words,

$$ \Pr(y \given x_1, \ldots, x_n) \propto \Pr(y) \prod_{i=1}^n \Pr(x_i \given y), $$

so, our classification rule becomes

$$ f(x_1, \ldots, x_n) = \argmax_{y \in \B} \Pr(y) \prod _{i=1}^n \Pr(x_i \given y).$$

Where $\propto$ means "proportional to" and  $\prod_{i = 1}^n g(i)$ is like summation $\left(\sum_{i=1}^n g(i)\right)$, except that addition is replaced with multiplication. For example,

$$\prod_{i = 1}^5 i^2 = 1^2 \cdot 2^2 \cdot 3^2 \cdot 4^2 \cdot 5^2.$$

**Note**: To estimate prior and conditional probabilities, we use the ratios of occurence counts found in the dataset. For example, to estimate $\Pr(x_1 = 0 \; \vert \; y = -1)$, we have to calculate the number of instances of class -1 for which $x_1 = 0$ and divide them by the number of instances of class -1.

**Note**: The naive independence assumption is usually false in practice for most features. Therefore, the resulting estimated probability distribution is usually a bad approximation of the true distribution. However, the resulting classifier often has a good performance, depending on the dataset.

### Quesiton 3.1 Estimating the Probability Distribution

It would be expensive to re-estimate prior and posterior probabilities every time, so we should save probabilities in memory.

Thus, you will need to save
1. $\Pr(y)$ for each $y \in \B$, and
2. $\Pr(x_i = u \; \vert \; y)$ for each $ i \in \{1, \ldots, n\}$, $u \in \mathbb{B}$ and $y \in \mathbb{B}$.

Remember that you are *estimating* the probabilities using the training set only.

In [45]:
### Answer starts here ###
counter_y = collections.Counter(y_train)
Pr_y = [counter_y[-1]/len(y_train), counter_y[1]/len(y_train)]

Pr_x = [[0 for i in range(num_features)] for j in range(4)]
for j in range(len(y_train)):
  for i in range(num_features):
    if y_train[j] == -1:
      if X_train_vect[j][i] == 0:
        Pr_x[0][i] = Pr_x[0][i] + 1/counter_y[-1]
      else:
        Pr_x[1][i] = Pr_x[1][i] + 1/counter_y[-1]
    else:
      if X_train_vect[j][i] == 0:
        Pr_x[2][i] = Pr_x[2][i] + 1/counter_y[1]
      else:
        Pr_x[3][i] = Pr_x[3][i] + 1/counter_y[1]
### Answer ends here ###

### Question 3.2 Creating the Naive Bayes Classifier

Create a function called `naive_bayes` which will take as input a list of features $x_1, \ldots, x_n$ and outputs the class with the largest posterior probability given the input features.

In [46]:
def naive_bayes(vec):
  ### Answer starts here ###
  good = Pr_y[1]
  bad = Pr_y[0]

  for i in range(len(vec)):
    if vec[i] == 0:
      good = good * Pr_x[2][i]
      bad = bad * Pr_x[0][i]
    else:
      good = good * Pr_x[3][i]
      bad = bad * Pr_x[1][i]
  if (good >= bad):
    return 1
  else:
    return -1
  
  ### Answer ends here ###

### Question 3.3 Measuring Performance

Using the naive Bayes classifier, predict the classes for each sample point in the training set as well as the test set and print accuracies.

**Note:** You should get train and test accuracies of about 80-85%.

**Hint:** You can use the `accuracy_score` function provided by `sklearn.metrics`

In [48]:
### Answer starts here ###
accuracy = 0
for i in range(len(X_train_vect)):
  if naive_bayes(X_train_vect[i]) == y_train[i]:
    accuracy += 1

print("Training accuracy = {}".format(accuracy/len(y_train)))

accuracy = 0
for i in range(len(X_test)):
  if naive_bayes(X_test[i]) == y_test[i]:
    accuracy += 1
print("Testing accuracy = {}".format(accuracy/len(y_test)))
### Answer ends here ###

Training accuracy = 0.8325533021320853
Testing accuracy = 0.8122724908996359


In [49]:
print(naive_bayes(preprocess_sample_point(
    'Terrible. Horrible. Boring. This movie is bad', vocabulary)))

print(naive_bayes(preprocess_sample_point(
    'This movie was pretty good', vocabulary)))

-1
1


## 4. Support Vector Machines

Quick recap of SVM: A support vector classifier tries to find the best separating hyperplane through the data. If the data is linearly separable, it finds a hyperplane that maximizes the margin. If it isn't, the classifier tries to minimize the cost associated with misclassifying points.



### Question 4.1 Creating a Support Vector Classifier
Using `scikit-learn`, create a support vector classifier for our review data.

1. Use `scikit-learn` to create a linear support vector classifer (name it `svm_clf`) (Make sure to use svm.LinearSVC and not svm.SVC as LinearSVC is more adapted to large datasets)
2. Fit the model to our training set
3. Print training accuracy
4. Print test accuracy

In [50]:
### Answer starts here ###
svm_clf = LinearSVC(dual=False)
svm_clf.fit(X_train_vect, y_train)
accuracy = 0
result_train = svm_clf.predict(X_train_vect)
for i in range(len(result_train)):
  if result_train[i] == y_train[i]:
    accuracy += 1
print(accuracy/len(X_train_vect))

accuracy = 0
result_test = svm_clf.predict(X_test)
for i in range(len(result_test)):
  if result_test[i] == y_test[i]:
    accuracy += 1
print(accuracy/len(X_test))

### Answer ends here ###


0.999879995199808
0.8295531821272851


In [51]:
print(svm_clf.predict([preprocess_sample_point(
    'Boring. Such a bad movie. It was terrible and predictable', vocabulary)]))

print(svm_clf.predict([preprocess_sample_point(
    'I really liked this movie, it\'s great!', vocabulary)]))

[-1]
[1]


## 5. Random Forests

Random forests are a type of ensemble classifier, i.e. they are made up of a number of 'weak' learners where the final classification is a combination of the classifications of each learner. 



### Question 5.1 Creating a Random Forest Classifier
Using `scikit-learn`, create a radom forest classifier for our review data.

1. Use `scikit-learn` to create a random forest classifier (name it `rfc`)
2. Fit the model to our training set
3. Print training accuracy
4. Print test accuracy



Be sure to check the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Try to play around with the hyperparameters to see if you can get higher accuracy. Specifically, try finding good values for `n_estimators`, `min_samples_split`, `max_depth` and `max_features`. Try to get accuracies close to the SVM's!

In [52]:
### Answer starts here ###
rfc = RandomForestClassifier()
rfc.fit(X_train_vect, y_train)
result_train = rfc.predict(X_train_vect)

accuracy = 0
for i in range(len(result_train)):
  if result_train[i] == y_train[i]:
    accuracy += 1
print(accuracy/len(y_train))

accuracy = 0
result_test = rfc.predict(X_test)
for i in range(len(result_test)):
  if result_test[i] == y_test[i]:
    accuracy += 1
print(accuracy/len(X_test))

### Answer ends here ###

1.0
0.828673146925877


In [ ]:
print(rfc.predict([preprocess_sample_point(
    'Boring. This movie is terrible', vocabulary)]))

print(rfc.predict([preprocess_sample_point(
    'This movie was pretty good', vocabulary)]))

### Question 5.2 Manual hyperparameter tuning

Tell us about your hyperparamter tuning in a few sentences! What was your approach? Which paramters did you try changing? Were you able to improve your accuracies? (no right answer, just tell us what you experiemented with!)

#############

 I'm going to be real honest here. The accuracy was really close to svm (although probably overfit) without any hyperparameters and it's midterm season. I'm going to leave it as is for now and come back in a week or 2 to play around with this and do the optional questions. Cheers!

#############

## 6. [Optional] Tuning hyperparameters with GridSearchCV
In this **optional** section, we will explore how to exhaustively tune the hyperparameters of a new classifier using `sklearn's` [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

### Question 6.1 Creating an Adaboost classifier
In this section, we will be tuning the hyperparameters of an `adaboost` model. 

As its `sklearn` [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) mentions, this classifier "begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases." 

For this question, simply create an instance of the classifer and name it `abc`.

In [ ]:
### Answer starts here ###

### Answer ends here ###

### Question 6.2 Finding the best parameters
In order to use GridSearchCV, we need to provide it with a classifier (here, `abc`) and lists of different values for the hyperparameters we want to tune. GridSearchCV will run a fit using each possible parameter and get crossvalidation scores. 

For this question, use the GridSearchCV documentation to:
1. Create a dict of parameters to tune with their respective list of values
2. Create an instance of GridSearchCV with `abc` as the estimator and the following arguments: `cv=5, refit=False, verbose=3`
3. Print the resulting scores and determine which parameters are best!


In [ ]:
### Answer starts here ###

### Answer ends here ###

### Question 6.3 Final result with best parameters
Using the best parameters found in the previous question, reinstantiate an `adaboost` classifier, fit it to the train data, and print the the accuracy score on the test set. 

In [ ]:
### Answer starts here ####

### Answer ends here ####

## 7. Recap and conclusion
Congratulations on reaching the end of Assignment 2! We hope you enjoyed it. 

Here's a recap of tasks and concepts explored in this assignment:


1.   **Preprocessing**
*   Cleaning train data: removing punctuation and HTML tags
*   Basic feature engineering: vectorizing data, bag-of-words representation 
*   Using learned vocabulary to preprocess test data

Preprocessing and data representation are very important in Natural Language Processing (NLP) projects. For more advanced preprocessing techniques, we highly recommend checking online the concepts of **stemming**, **n-gramming**, **stopwords removal**!

2.   **Naive Bayes**
*   Naive Bayes: recap of theory
*   Implementing model from scratch

Find out more about Naive Bayes implementations in `sklearn`: [GaussianNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html), [MultinomialNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html), and more!

3.   **Support Vector Machine**
*   SVM: brief recap of theory and testing
*   First introduction to `sklearn`: using already implemented models to classify data

4.   **Random Forests**
*   Random Forests: brief recap of theory and testing
*   Introduction to hyperparameter tuning

The default parameters are not always good enough! Find out more about how to tune hyperparameters in `sklearn` [here](https://scikit-learn.org/stable/modules/grid_search.html) or do the optional section 6 of this assignment. 

5.   **Tuning Adaboost with GridSearchCV**
*   Adaboost: bried recap of theory and testing 
*   Tuning hyperparameters using GridSearchCV

## 8. Submission

To submit your work, copy this colab, set the permissions of your copy to "anyone with the link can view" and copy paste the link to your colab on MyCourses submission.